In [1]:
from NPSmethods2 import*
import numpy as np
import glob
import pickle as pkl
import tensorflow as tf

In [2]:
def calcNoisePowSpecs(ODs):

    noisePowSpecs = []
    for k in range(len(ODs)):
        noise = ODs[k]
        noiseFFT = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(noise)))
        noisePowSpec = np.abs(noiseFFT)**2
        noisePowSpec = noisePowSpec / ODs[k].sum()
        noisePowSpec[noisePowSpec.shape[0]//2, noisePowSpec.shape[1]//2] = 0
    
        noisePowSpecs.append(noisePowSpec)

    return noisePowSpecs

In [3]:
imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
}

choices = {
    "if_Save"                     : False,
    "do_Fit"                      : True,
    "plot_2dGas"                  : False,
    "plot_NoisePowSpec"           : False,
    "plot_NoisePowSpec_LineCut"   : False,
    "plot_Pupil"                  : False,
    "plot_PSF"                    : False,
    "plot_PSF_LineCut"            : False,
    "plot_PSF_abs2"               : False,
    "plot_PSF_abs2_LineCut"       : False,
    "plot_Sk"                     : False,
    "normalize"                   : False
}

path = r'C:\Users\eric0\Jupyter Notebooks\Cold Physics\raw_image'
savepath = r'C:\Users\eric0\Jupyter Notebooks\Cold Physics\results'
numOfImgsInEachRun = 1
rampingParameter = 0
trapRegion = (slice(100, 200), slice(100, 200))
noiseRegion = (slice(0, 300), slice(0, 300))

In [4]:
parameters = []
images = []

for folder in glob.iglob(path + "\\*"):
    atomODs, atomODAvg, _, _, _, _ = \
        readInImages(folder + "\\", numOfImgsInEachRun, rampingParameter, trapRegion, noiseRegion)
    noisePowSpecs = calcNoisePowSpecs(atomODs)
    _, _, _, _, popt, _ = doCalibration(folder, savepath, trapRegion, noiseRegion, numOfImgsInEachRun, rampingParameter, imgSysData, choices)
    for noisePowSpec in noisePowSpecs:
        images.append(noisePowSpec)
    for i in range(len(noisePowSpecs)):
        parameters.append(popt)

tau     =  1.0000 : describe the decaying of transmission efficiency with radius
S0      =  0.0100 : spherical aberration
alpha   =  2.4600 : astigmatism
beta    =  0.3640 : defocus
delta_s = -0.0379 : phase imprint by atom scattering

tau     =  1.0000 : describe the decaying of transmission efficiency with radius
S0      =  0.0100 : spherical aberration
alpha   =  2.3192 : astigmatism
beta    =  0.5329 : defocus
delta_s = -0.0325 : phase imprint by atom scattering

tau     =  1.0000 : describe the decaying of transmission efficiency with radius
S0      =  0.0000 : spherical aberration
alpha   =  2.4600 : astigmatism
beta    = -2.0000 : defocus
delta_s = -0.1055 : phase imprint by atom scattering

tau     =  1.0000 : describe the decaying of transmission efficiency with radius
S0      =  0.0000 : spherical aberration
alpha   =  2.4600 : astigmatism
beta    = -1.7309 : defocus
delta_s = -0.0049 : phase imprint by atom scattering

tau     =  1.0000 : describe the decaying of transmissio

In [5]:
images = np.delete(images, np.s_[74:100], axis = 1)
images = np.delete(images, np.s_[0:25], axis = 1)
images = np.delete(images, np.s_[74:100], axis = 2)
images = np.delete(images, np.s_[0:25], axis = 2)

In [6]:
print(np.shape(images))

(364, 49, 49)


In [7]:
for imnum in range(len(images)):
    K_x, K_y, _, _ = getFreq(imgSysData["CCDPixelSize"], imgSysData["magnification"], (50,50))
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, images[imnum], cmap=cm.jet, vmin = 0, vmax = images[imnum].max())
    plt.colorbar(im)
    fig.savefig("C:\\Users\\eric0\\OneDrive\\Desktop\\Cold Physics\\real_NPS_ims\\real_NPS_%d" % imnum)
    plt.close(fig)

In [8]:
images = np.array(images).reshape((len(images),-1))
parameters = np.array(parameters)
print(np.argwhere(np.amin(images, axis=1) < 0))
parameters = np.delete(parameters, np.where(np.amin(images, axis=1) < 0), 0)
images = np.delete(images, np.where(np.amin(images, axis=1) < 0), 0)

[[257]]


In [11]:
defocus = parameters[:,5]
threeimages = []
fiveimages = []
threeimparams = []
fiveimparams = []
for i in range(len(images)):
    if (i + 1) % 3 == 0:
        if defocus[i] == defocus[i - 2]:
            threeimages.append(sum(images[i - 2:i] / 3))
            threeimparams.append(defocus[i])
                               
    if (i + 1) % 5 == 0:
        if defocus[i] == defocus[i - 4]:
            fiveimages.append(sum(images[i - 4:i] / 5))
            fiveimparams.append(defocus[i])
threeimages = np.array(threeimages)
fiveimages = np.array(fiveimages)

In [12]:
# xscaler = pkl.load(open(savepath + "\\xscaler", "rb"))
# images = xscaler.transform(images)
images /= 20
threeimages /= 20
fiveimages /= 20
# mean = pkl.load(open(savepath + "\\mean", "rb"))
# std = pkl.load(open(savepath + "\\std", "rb"))
# images -= mean
# images /= std
# threeimages -= mean
# threeimages /= std
# fiveimages -= mean
# fiveimages /= std
for i in range(len(images)):
    images[i] -= np.mean(images[i])
    images[i] /= np.std(images[i])
for i in range(len(threeimages)):
    threeimages[i] -= np.mean(threeimages[i])
    threeimages[i] /= np.std(threeimages[i])
for i in range(len(fiveimages)):
    fiveimages[i] -= np.mean(fiveimages[i])
    fiveimages[i] /= np.std(fiveimages[i])

In [15]:
print(np.max(images))
print(np.min(images))

30.163363440762478
-0.7050754180779927


In [16]:
images = tf.convert_to_tensor(images)
threeimages = tf.convert_to_tensor(threeimages)
fiveimages = tf.convert_to_tensor(fiveimages)
parameters = tf.convert_to_tensor(parameters)
threeimparams = tf.convert_to_tensor(threeimparams)
fiveimparams = tf.convert_to_tensor(fiveimparams)

In [17]:
pkl.dump(images, open(savepath + "\\realdatain", "wb"))
pkl.dump(threeimages, open(savepath + "\\3images", "wb"))
pkl.dump(fiveimages, open(savepath + "\\5images", "wb"))
pkl.dump(parameters, open(savepath + "\\realdataout", "wb"))
pkl.dump(threeimparams, open(savepath + "\\3imparams", "wb"))
pkl.dump(fiveimparams, open(savepath + "\\5imparams", "wb"))